In [27]:
import json
import re
import pandas as pd
import numpy as np

def clean_text(value):
    """Clean text by fixing encoding issues and normalizing spaces"""
    if not isinstance(value, str):
        return value
    
    # Fix common encoding issues - expanded list for French characters
    replacements = {
        'Ã©': 'é', 'Ã¨': 'è', 'Ã´': 'ô', 'Ã¹': 'ù',
        'Ã': 'à', 'Ã®': 'î', 'Ã»': 'û', 'Ã§': 'ç',
        'Ã€': 'À', 'Ã‰': 'É', 'ÃŠ': 'Ê', 'Ã"': 'Ô',
        'Â': '', '\xa0': ' ', '\u202f': ' ', '\n': ' ', '\t': ' '
    }
    
    # Apply all replacements
    for old, new in replacements.items():
        value = value.replace(old, new)
    
    # Normalize spaces and strip whitespace
    value = ' '.join(value.split())
    return value.strip()

def clean_price(price):
    """Convert price string to numeric value by removing TND and formatting"""
    if pd.isna(price) or price in [None, '']:
        return None
    
    if not isinstance(price, str):
        try:
            return float(price)
        except (ValueError, TypeError):
            return None
    
    # Remove all non-numeric characters except dots and commas
    price = re.sub(r'[^\d,.]', '', price)
    
    # Handle thousand separators and decimal points
    if ',' in price and '.' in price:
        if price.find(',') > price.find('.'):  # Comma is decimal (1.000,50)
            price = price.replace('.', '').replace(',', '.')
        else:  # Dot is decimal (1,000.50)
            price = price.replace(',', '')
    elif ',' in price:
        # Treat as thousand separator or decimal based on position
        if len(price.split(',')[-1]) == 3 and len(price.split(',')) == 2:
            price = price.replace(',', '')  # Thousand separator
        else:
            price = price.replace(',', '.')  # Decimal separator
    
    try:
        return float(price)
    except (ValueError, TypeError):
        return None

def extract_floor_level(floor_level_text):
    """Extract the floor level number from text like '4Ã¨me' or '2ème étage'"""
    if pd.isna(floor_level_text) or floor_level_text in [None, '']:
        return None
    
    # Clean the text first to handle encoding issues
    floor_level_text = clean_text(floor_level_text)
    
    # Pattern to match digits at the beginning of the string
    # This will extract "4" from "4ème" or "4e étage"
    match = re.search(r'^(\d+)', floor_level_text)
    if match:
        return int(match.group(1))
    
    # Check for textual floor levels
    lower_text = floor_level_text.lower()
    if any(term in lower_text for term in ['rez', 'rdc', 'ground']):
        return 0
    elif any(term in lower_text for term in ['premier', '1er', 'first']):
        return 1
    elif any(term in lower_text for term in ['deuxième', '2e', '2ème', 'second']):
        return 2
    elif any(term in lower_text for term in ['troisième', '3e', '3ème', 'third']):
        return 3
    elif any(term in lower_text for term in ['quatrième', '4e', '4ème', 'fourth']):
        return 4
    elif any(term in lower_text for term in ['cinquième', '5e', '5ème', 'fifth']):
        return 5
    
    # Look for any number in the text as a last resort
    match = re.search(r'(\d+)', floor_level_text)
    if match:
        return int(match.group(1))
    
    return None

def is_invalid_row(row):
    """Check if a row contains placeholder or empty values"""
    invalid_indicators = {
        'ad_id': ['No Ad ID', ''],
        'title': ['No Title', ''],
        'image_url': ['No Image', ''],
        'location': ['No Location', ''],
        'price': ['No Price', '']
    }
    
    # Check for placeholder values in key fields
    for field, invalid_values in invalid_indicators.items():
        if field in row and str(row[field]) in invalid_values:
            return True
    
    # Check if all amenity columns are 0 or missing
    amenity_cols = [col for col in row.keys() if col.startswith(('has_', 'amenities_'))]
    if amenity_cols and all(row.get(col, 0) == 0 for col in amenity_cols):
        return True
        
    return False

def transform_property_data(input_data, is_file=True):
    """Transform raw property data into structured format"""
    # Load input data
    if is_file:
        with open(input_data, 'r', encoding='utf-8') as file:
            data_list = json.load(file)
    else:
        data_list = json.loads(input_data)
        if not isinstance(data_list, list):
            data_list = [data_list]

    # Define output columns
    columns = [
        # Core fields
        'title', 'price', 'location', 'link', 'ad_id', 'image_url', 'scraped_at',
        # Property characteristics
        'property_type', 'property_condition', 'property_age', 'floor_level',
        'orientation', 'floor_material', 'property_standing', 'construction_status',
        'delivery_date',
        # Features
        'area_sqm', 'total_rooms', 'bedrooms', 'bathrooms',
        # Amenities
        'has_garden', 'has_terrace', 'has_garage', 'has_elevator', 'has_pool',
        'has_concierge', 'has_air_conditioning', 'has_heating', 'has_security',
        'has_double_glazing', 'has_armored_door', 'has_equipped_kitchen',
        'has_fireplace', 'has_furniture', 'has_sea_view', 'has_mountain_view',
        'has_parabolic_antenna', 'has_european_living_room', 'has_storage_room',
        'has_fridge', 'has_oven', 'has_washing_machine', 'has_microwave'
    ]

    # Field mappings from French to English
    field_mapping = {
        'Type de bien': 'property_type',
        'Etat': 'property_condition',
        'État': 'property_condition',
        'Etat du bien': 'property_age',
        'Étage du bien': 'floor_level',
        'Etage du bien': 'floor_level',
        'Orientation': 'orientation',
        'Type du sol': 'floor_material',
        'Standing': 'property_standing',
        'État': 'construction_status',
        'Etat': 'construction_status',
        'Livraison:': 'delivery_date'
    }

    # Amenity mappings
    amenities_mapping = {
        'Jardin': 'has_garden',
        'Terrasse': 'has_terrace',
        'Garage': 'has_garage',
        'Ascenseur': 'has_elevator',
        'Piscine': 'has_pool',
        'Concierge': 'has_concierge',
        'Climatisation': 'has_air_conditioning',
        'Chauffage central': 'has_heating',
        'Sécurité': 'has_security',
        'Double vitrage': 'has_double_glazing',
        'Porte blindée': 'has_armored_door',
        'Cuisine équipée': 'has_equipped_kitchen',
        'Cheminée': 'has_fireplace',
        'Meublé': 'has_furniture',
        'Vue sur mer': 'has_sea_view',
        'Vue sur les montagnes': 'has_mountain_view',
        'Antenne parabolique': 'has_parabolic_antenna',
        'Salon européen': 'has_european_living_room',
        'Chambre rangement': 'has_storage_room',
        'Réfrigérateur': 'has_fridge',
        'Four': 'has_oven',
        'Machine à laver': 'has_washing_machine',
        'Micro-ondes': 'has_microwave'
    }

    transformed_data = []
    additional_amenities = set()

    for property_data in data_list:
        row = {col: None for col in columns}

        # Process core fields
        for field in ['title', 'location', 'link', 'ad_id', 'image_url', 'scraped_at']:
            if field in property_data:
                row[field] = clean_text(property_data[field]) if isinstance(property_data[field], str) else property_data[field]
        
        # Clean and convert price
        if 'price' in property_data:
            row['price'] = clean_price(property_data['price'])

        # Process detailed characteristics
        if 'detailed_characteristics' in property_data:
            for fr_field, en_field in field_mapping.items():
                if fr_field in property_data['detailed_characteristics']:
                    value = property_data['detailed_characteristics'][fr_field]
                    
                    if en_field == 'floor_level':
                        row[en_field] = extract_floor_level(value)
                    else:
                        row[en_field] = clean_text(value) if isinstance(value, str) else value

        # Process features
        if 'features' in property_data:
            features = property_data['features']
            
            # Area conversion
            if 'area' in features:
                area = features['area']
                if isinstance(area, str):
                    area = clean_text(area)
                    match = re.search(r'(\d+)', area)
                    row['area_sqm'] = float(match.group(1)) if match else None
            
            # Room counts
            for room_type in ['total_rooms', 'bedrooms', 'bathrooms']:
                if room_type in features:
                    room_count = features[room_type]
                    if isinstance(room_count, str):
                        room_count = clean_text(room_count)
                        match = re.search(r'(\d+)', room_count)
                        row[room_type] = int(match.group(1)) if match else None

        # Initialize amenities
        for amenity_col in [col for col in columns if col.startswith('has_')]:
            row[amenity_col] = 0

        # Process amenities
        if 'amenities' in property_data:
            for amenity in property_data['amenities']:
                clean_amenity = clean_text(amenity)
                if clean_amenity in amenities_mapping:
                    row[amenities_mapping[clean_amenity]] = 1
                else:
                    amenity_col = 'amenities_' + clean_amenity.lower().replace(' ', '_')
                    additional_amenities.add(amenity_col)
                    row[amenity_col] = 1

        # Only keep valid rows
        if not is_invalid_row(row):
            transformed_data.append(row)

    # Ensure all additional amenities are present in all rows
    for row in transformed_data:
        for amenity in additional_amenities:
            if amenity not in row:
                row[amenity] = 0

    return transformed_data, additional_amenities

def organize_columns(df):
    """Organize columns into logical groups"""
    column_groups = [
        # Group 1: Identifiers
        ['ad_id', 'title', 'link', 'image_url', 'scraped_at'],
        # Group 2: Location
        ['location'],
        # Group 3: Price
        ['price'],
        # Group 4: Property details
        ['property_type', 'property_condition', 'property_age', 'construction_status',
         'delivery_date', 'property_standing'],
        # Group 5: Physical characteristics
        ['area_sqm', 'total_rooms', 'bedrooms', 'bathrooms', 'floor_level',
         'orientation', 'floor_material'],
        # Group 6: Common amenities
        ['has_garden', 'has_terrace', 'has_garage', 'has_elevator', 'has_pool',
         'has_air_conditioning', 'has_heating', 'has_equipped_kitchen']
    ]
    
    # Group 7: Other amenities
    other_amenities = sorted([col for col in df.columns 
                            if col.startswith('has_') and col not in column_groups[5]])
    
    # Group 8: Additional amenities
    additional_amenities = sorted([col for col in df.columns 
                                 if col.startswith('amenities_')])
    
    # Combine all ordered columns
    ordered_columns = []
    for group in column_groups:
        ordered_columns.extend([col for col in group if col in df.columns])
    ordered_columns.extend(other_amenities)
    ordered_columns.extend(additional_amenities)
    
    # Add any remaining columns
    remaining_cols = [col for col in df.columns if col not in ordered_columns]
    ordered_columns.extend(remaining_cols)
    
    return ordered_columns

def main(input_file_path):
    """Main processing function"""
    # Transform the data
    properties, additional_amenities = transform_property_data(input_file_path)
    df = pd.DataFrame(properties)
    
    # Clean up missing values
    df.replace({np.nan: None}, inplace=True)
    
    # Organize columns
    ordered_columns = organize_columns(df)
    df = df[ordered_columns]
    
    # Save to CSV
    df.to_csv('mubwab_data.csv', index=False, encoding='utf-8')
    
    # Print summary
    print("Data transformation complete!")
    print(f"Saved {len(df)} properties to 'mubwab_data.csv'")
    print(f"Total columns: {len(df.columns)}")
    
    return df

if __name__ == "__main__":
    input_file = 'mubawab_properties_completed.json'
    main(input_file)

Data transformation complete!
Saved 8256 properties to 'mubwab_data.csv'
Total columns: 45
